**Avant de lancer ce notebook merci de générer les csv à l'aide du [notebook suivant](Create-csv-files.ipynb).**

In [1]:
# Commenter la configuration non utilisée

# Configuration production
mongoDBUrl = 'mongodb017.westeurope.cloudapp.azure.com'
port = 32002

# Configuration dev
#mongoDBUrl = 'localhost'
#port = 27017

In [2]:
import pandas as pd
import numpy as np
import json
from pymongo import MongoClient

In [3]:
client = MongoClient( mongoDBUrl, port )
db = client['db_credit']

In [4]:
def drop_collections(client, db):
    
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop()

In [5]:
drop_collections(client, db)

Dénormalisation retenue:

In [6]:
{
    "members":
    {
        "member_no": "1",
        "lastname": "DOUGLAS",
        "firstname": "Michael",
        "phone_number":"blabla",

        "location":
        {
            "country":"France",
            "region_no":"IDF",
            "city":"Paris",
            "mail_code":"75000",
            "street":"blabla"
        },

        "balance":
        {
            "prev_balance": "1000",
            "curr_balance": "1200"
        },

        "_comment":"No need to have a list, each employee have only 1 corporation",
        "corporation":
        {
            "corp_no":1,
            "corp_name":"blabla",
            "region_no":"XX"
        },

        "charges": [
            {
                "charge_no": "125",
                "category_no": "12",
                "category_desc": "Travel",
                "charge_dt": "2000-02-22 10:43:38",
                "charge_amount": "336.0000",

                "provider_no":12,
                "provider_name":"BNP Paribas",
                "provider_phone_number": "blabla",
                "payments":[1546, 1547,... , 1578]
            }
        ]
    },
     "providers":
        {
            "provider_no":1,
            "provider_name":"BNP",
            "location":
            {
                "country":"France",
                "region_no":"IDF",
                "city":"Paris",
                "mail_code":"75000"
            }
        },
 
 
    "payments":
        {
            "payment_no":1546,
            "payment_dt":"2000-02-22 10:43:38",
            "payment_amt":125.00
        }
}


{'members': {'member_no': '1',
  'lastname': 'DOUGLAS',
  'firstname': 'Michael',
  'phone_number': 'blabla',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000',
   'street': 'blabla'},
  'balance': {'prev_balance': '1000', 'curr_balance': '1200'},
  '_comment': 'No need to have a list, each employee have only 1 corporation',
  'corporation': {'corp_no': 1, 'corp_name': 'blabla', 'region_no': 'XX'},
  'charges': [{'charge_no': '125',
    'category_no': '12',
    'category_desc': 'Travel',
    'charge_dt': '2000-02-22 10:43:38',
    'charge_amount': '336.0000',
    'provider_no': 12,
    'provider_name': 'BNP Paribas',
    'provider_phone_number': 'blabla',
    'payments': [1546, 1547, Ellipsis, 1578]}]},
 'providers': {'provider_no': 1,
  'provider_name': 'BNP',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000'}},
 'payments': {'payment_no': 1546,
  'payment_dt': '2000-02-22 10:43:3

In [7]:
Members = pd.read_csv('Members.csv')
Corporations = pd.read_csv('Corporations.csv')
Categories = pd.read_csv('Categories.csv')
Providers = pd.read_csv('Prodivers.csv')
Charges = pd.read_csv('Charges.csv')
Statements = pd.read_csv('Statements.csv')
Payments = pd.read_csv('Payments.csv')

In [8]:
Charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
0,1,5000,24,2020-11-12,3020,73,0
1,0,40667,36,2021-07-03,4273,69,1
2,0,9638,48,2020-07-26,3992,42,2
3,2,5336,12,2020-09-11,9315,6,3
4,1,5000,60,2021-06-13,2008,188,4
...,...,...,...,...,...,...,...
49995,1,5000,48,2021-01-01,7063,58,49995
49996,0,5000,60,2021-07-30,3495,78,49996
49997,1,52963,48,2021-08-19,9286,86,49997
49998,2,49051,60,2020-09-14,1859,143,49998


In [9]:
Payments

,payment_amt,due_dt,charge_no,member_no,payment_no
0,200.791667,2020-08-26,2,3992,60
1,18.729167,2020-08-15,6,4765,192
2,554.883333,2020-08-10,8,2095,276
3,92.750000,2020-04-30,15,5867,552
4,92.750000,2020-05-30,15,5867,553
...,...,...,...,...,...
57616,67.979167,2020-08-28,49990,4431,1798104
57617,416.666667,2020-05-17,49993,9351,1798212
57618,416.666667,2020-06-17,49993,9351,1798213
57619,416.666667,2020-07-17,49993,9351,1798214


In [10]:
member_no = 907

charges = Charges[Charges.member_no == member_no]
charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
157,1,5000,48,2021-06-17,907,4,157
4990,2,7038,12,2021-11-21,907,177,4990
39553,1,15127,48,2020-09-26,907,63,39553
42062,2,4316,48,2021-03-04,907,91,42062
43629,0,5000,24,2020-06-19,907,63,43629


In [11]:
providers_no = charges.provider_no.values
providers_no

array([  4, 177,  63,  91,  63])

In [12]:
providers = []
for provider_no in providers_no:
    provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
    providers.append(provider)
providers

[provider_no                             4
 provider_name              Morgan Stanley
 phone_number                   6101917439
 country                     United States
 region_no                              MD
 city                            Annapolis
 mail_code                           21401
 street           992 Sherwood Forest Road
 Name: 4, dtype: object,
 provider_no                            177
 provider_name                      US Bank
 phone_number                     667093979
 country                      United States
 region_no                               OK
 city                          Midwest City
 mail_code                            73110
 street           319 West Silverwood Drive
 Name: 177, dtype: object,
 provider_no                             63
 provider_name                Goldman Sachs
 phone_number                    5843429356
 country                      United States
 region_no                               DC
 city                           

In [13]:
charges_no = charges.charge_no.values
charges_no

array([  157,  4990, 39553, 42062, 43629])

In [14]:
payments = []
for charge_no in charges_no:
    payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
    payments.append(list(payments_no))
payments

[[], [], [], [], [1569420, 1569421]]

In [15]:
charges_json = []
for i in range(len(charges_no)):
    charge = charges.iloc[i-1]
    provider = providers[i]
    payments_list = payments[i]
    charge_json = {
        "charge_no": charge.charge_no,
        "category_no": charge['Charge type'],
        #"category_desc": "Travel",
        "charge_dt": charge['dt'],
        "charge_amount": charge['Charge amount'],

        "provider_no": provider.provider_no,
        "provider_name": provider.provider_name,
        "provider_phone_number": provider.phone_number,
        "payments": payments_list
    }
    charges_json.append(charge_json)
    
charges_json

[{'charge_no': 43629,
  'category_no': 0,
  'charge_dt': '2020-06-19',
  'charge_amount': 5000,
  'provider_no': 4,
  'provider_name': 'Morgan Stanley',
  'provider_phone_number': 6101917439,
  'payments': []},
 {'charge_no': 157,
  'category_no': 1,
  'charge_dt': '2021-06-17',
  'charge_amount': 5000,
  'provider_no': 177,
  'provider_name': 'US Bank',
  'provider_phone_number': 667093979,
  'payments': []},
 {'charge_no': 4990,
  'category_no': 2,
  'charge_dt': '2021-11-21',
  'charge_amount': 7038,
  'provider_no': 63,
  'provider_name': 'Goldman Sachs',
  'provider_phone_number': 5843429356,
  'payments': []},
 {'charge_no': 39553,
  'category_no': 1,
  'charge_dt': '2020-09-26',
  'charge_amount': 15127,
  'provider_no': 91,
  'provider_name': 'PNC Financial Services',
  'provider_phone_number': 3884400947,
  'payments': []},
 {'charge_no': 42062,
  'category_no': 2,
  'charge_dt': '2021-03-04',
  'charge_amount': 4316,
  'provider_no': 63,
  'provider_name': 'Goldman Sachs',
  

In [16]:
member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
member

member_no                    907
lastname                 Jackson
firstname                Heather
phone_number           677432617
country            United States
region_no                     CA
city                     Oakland
mail_code                  94605
street          2747 68th Avenue
corp_no                      241
prev_balance         1256.508338
curr_balance          1101.50097
Name: 907, dtype: object

In [17]:
corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
corporation

corp_no                                 241
corp_name       Brown, Sanders and Hamilton
phone_number                     3292134079
country                       United States
region_no                                MA
city                              Brookline
mail_code                              2446
street                     19 Euston Street
Name: 241, dtype: object

In [18]:
member_json = {
    "member_no": member_no,
    "lastname": member.lastname,
    "firstname": member.firstname,
    "phone_number": member.phone_number,

    "location":
    {
        "country": member.country,
        "region_no": member.region_no,
        "city": member.city,
        "mail_code": member.mail_code,
        "street": member.street
    },

    "balance":
    {
        "prev_balance": member.prev_balance,
        "curr_balance": member.curr_balance
    },
    "corporation":
    {
        "corp_no": corporation.corp_no,
        "corp_name": corporation.corp_name,
        "region_no": corporation.region_no
    },

    "charges": charges_json
}
member_json

{'member_no': 907,
 'lastname': 'Jackson',
 'firstname': 'Heather',
 'phone_number': 677432617,
 'location': {'country': 'United States',
  'region_no': 'CA',
  'city': 'Oakland',
  'mail_code': 94605,
  'street': '2747 68th Avenue'},
 'balance': {'prev_balance': 1256.508337530429,
  'curr_balance': 1101.5009695834951},
 'corporation': {'corp_no': 241,
  'corp_name': 'Brown, Sanders and Hamilton',
  'region_no': 'MA'},
 'charges': [{'charge_no': 43629,
   'category_no': 0,
   'charge_dt': '2020-06-19',
   'charge_amount': 5000,
   'provider_no': 4,
   'provider_name': 'Morgan Stanley',
   'provider_phone_number': 6101917439,
   'payments': []},
  {'charge_no': 157,
   'category_no': 1,
   'charge_dt': '2021-06-17',
   'charge_amount': 5000,
   'provider_no': 177,
   'provider_name': 'US Bank',
   'provider_phone_number': 667093979,
   'payments': []},
  {'charge_no': 4990,
   'category_no': 2,
   'charge_dt': '2021-11-21',
   'charge_amount': 7038,
   'provider_no': 63,
   'provider_na

In [19]:
P = []

In [20]:
def get_charges(member_no):
    charges = Charges[Charges.member_no == member_no]
    return charges

def get_member(member_no):
    member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
    return member

def get_providers(charges):
    providers_no = charges.provider_no.values
    providers = []
    for provider_no in providers_no:
        provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
        providers.append(provider)
    return providers

def get_payments(charges):
    charges_no = charges.charge_no.values
    payments = []
    for charge_no in charges_no:
        payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
        payments.append(list(payments_no))
    return payments

def get_charges_json(member_no):
    charges = get_charges(member_no)
    providers = get_providers(charges)
    payments = get_payments(charges)
    
    charges_json = []
    for i in range(len(charges)):
        charge = charges.iloc[i-1]
        provider = providers[i]
        payments_list = payments[i]
        json = {
            "charge_no": charge.charge_no,
            "category_no": charge['Charge type'],
            "category_desc": get_category_name(charge['Charge type']),
            "charge_dt": charge['dt'],
            "charge_amount": charge['Charge amount'],

            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "provider_phone_number": provider.phone_number,
            "provider_city": provider.city,
            "payments": payments_list
        }
        charges_json.append(json)

    return charges_json

def get_category_name(category_no):
    return Categories.loc[lambda x: x.category_no == category_no].category_name.loc[category_no]

def get_corporation(member):
    corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
    return corporation

def get_member_json(member_no):
    charges_json = get_charges_json(member_no)
    member = get_member(member_no)
    corporation = get_corporation(member)
    
    member_json = {
        "member_no": member_no,
        "lastname": member.lastname,
        "firstname": member.firstname,
        "phone_number": member.phone_number,

        "location":
        {
            "country": member.country,
            "region_no": member.region_no,
            "city": member.city,
            "mail_code": member.mail_code,
            "street": member.street
        },
        
        "balance":
        {
            "prev_balance": member.prev_balance,
            "curr_balance": member.curr_balance
        },

        
        "corporation":
        {
            "corp_no": corporation.corp_no,
            "corp_name": corporation.corp_name,
            "region_no": corporation.region_no
        },

        "charges": charges_json
    }
    return member_json

def get_members_json():
    return Members.member_no.apply(lambda x: get_member_json(x)).to_list()
    # On utilise .uniques() pour éviter les doublons

In [21]:
def get_providers_json():
    providers = [
        {
            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "location":
            {
                "country": provider.country,
                "region_no": provider.region_no,
                "city": provider.city,
                "mail_code": provider.mail_code
            }
        } for (_,provider) in Providers.iterrows()]
    return providers

def get_payments_json():
    payments = [{
            "payment_no": payment.payment_no,
            "payment_dt": payment.due_dt,
            "payment_amt": payment.payment_amt
        } for (_,payment) in Payments.iterrows()]
    return payments

def final_json():
    return {
        'members': get_members_json(),
        'providers': get_providers_json(),
        'payments': get_payments_json()
    }

# Class pour résoudre les problèmes d'encoding entre numpy et le module json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def _export_mongo(data, filename, collection_name):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, cls=NpEncoder, indent=4)

    client = MongoClient( mongoDBUrl, port )
    db = client['db_credit']
    collection = db[collection_name]

    with open(filename) as f:
        file_data = json.load(f)
        collection.insert_many(file_data)
        client.close()
        
def members_export_mongo():
    data = get_members_json()
    filename = 'members.json'
    collection_name = 'members'
    _export_mongo(data, filename, collection_name)
    
def providers_export_mongo():
    data = get_providers_json()
    filename = 'providers.json'
    collection_name = 'providers'
    _export_mongo(data, filename, collection_name)
    
def payments_export_mongo():
    data = get_payments_json()
    filename = 'payments.json'
    collection_name = 'payments'
    _export_mongo(data, filename, collection_name)
    
def export_mongo():
    members_export_mongo()
    providers_export_mongo()
    payments_export_mongo()

In [22]:
export_mongo()

In [23]:
client = MongoClient( mongoDBUrl, port )
db = client['db_credit']
db.list_collection_names()

['providers', 'payments', 'members']

In [24]:
collection = db['members']
collection.count_documents({})

10000

In [25]:
collection = db['payments']
collection.count_documents({})

57621

In [26]:
collection = db['providers']
collection.count_documents({})

200

In [27]:
client.close()